In [1]:
!pip install --quiet bnlp_toolkit
# !pip install git+https://github.com/csebuetnlp/normalizer
!pip install bnunicodenormalizer

  Preparing metadata (setup.py) ... - \ done
  Created wheel for bnunicodenormalizer: filename=bnunicodenormalizer-0.1.6-py3-none-any.whl size=22795 sha256=a2012fe3d830f4094d4ddfb3e49669ee9b69b6305c0be504842c09542e0828c5
  Stored in directory: /root/.cache/pip/wheels/f4/d7/e9/16732a619cbf5a63fdc9f6e2f9eb5fcf73fa023735237330e9
Successfully built bnunicodenormalizer


In [2]:
import os
import ast
import re
import json
import pandas as pd
import numpy as np
import string
import matplotlib.pyplot as plt

from bnlp import BasicTokenizer
# from normalizer import normalize
from bnunicodenormalizer import Normalizer


In [3]:
def get_logger(filename='train_log'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()

In [4]:
data_path= "/kaggle/input/bengaliai-speech/train.csv"
df= pd.read_csv(data_path)
df= df[df['split']=='valid'].reset_index(drop= True)
df.head()

,id,sentence,split
0,0000e711c2b1,তিনি এবং তাঁর মা তাদের পৈতৃক বাড়িতে থেকে প্রত...,valid
1,00036c2a2d9d,কৃত্তিবাস রামায়ণ-বহির্ভূত অনেক গল্প এই অনুবাদ...,valid
2,00065e317123,তিনি তার সুশৃঙ্খল সামরিক বাহিনী এবং সুগঠিত শাস...,valid
3,00065f40df52,তিনি বিজয়নগর সাম্রাজ্যের বিরুদ্ধে এবং বিজাপুর...,valid
4,0009b022c8ea,এটি মূলত একটি মরুময় অঞ্চল।,valid


In [5]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [6]:
# # punctuations= string.punctuation + '।'
# punctuations= [',', '।', '?', '!', '-', ';', ':']
# def check_pun(text):
#     for p in punctuations:
#         if p in text:
#             return 1
#     return 0
# def extract_punctuation(text):
#     pattern = r'\s*([' + re.escape(''.join(punctuations)) + '])'
#     punctuation_matches = re.findall(pattern, text)
#     punctuation_list = list(punctuation_matches)
#     return punctuation_list

# def remove_duplicate_punctuation(text):
#     # remove consecutive occurrences of the same punctuation
#     pattern = r'\s*([' + re.escape(''.join(punctuations)) + '])'
# #     pattern = r'([,!?;:।-_.])\1+'
#     text = re.sub(pattern, r'\1', text)
#     return text
# def remove_space_before_punctuation(text):
#     # remove spaces before punctuation
#     pattern = r'\s*([' + re.escape(''.join(punctuations)) + '])'
#     #r'([{}]-)'.format(re.escape(pun))
#     text = re.sub(pattern, r'\1', text)
#     if text[-1] == ' ':
#         text= text[:-1]
#     return text
# def add_space_after_punctuation(text):
#     punctuations= [',', '।', '?', '!', '-', ';', ':']
#     for pun in punctuations:
#         text= text.replace(pun, pun + ' ')
#     return text


In [7]:
# add_space_after_punctuation('কৃত্তিবাস রামায়ণ-বহির্ভূত অনেক গল্প এই')
# add_space_after_punctuation(remove_space_before_punctuation("দক্ষিণ - পূর্ব রেলের,, মেদিনীপুর-আর্দ্রা শাখায় এই স্টেশন ? অবস্থিত।"))

modified_sentence: add space for every punctuations 

In [8]:
# df['pun']= df['sentence'].apply(lambda x: check_pun(x))
# df['punctuations']= df['sentence'].apply(lambda x: extract_punctuation(x))
# df= df[df['pun']==1].reset_index(drop= True)
# df['modified_sentence']= df['sentence'].apply(lambda x: add_space_after_punctuation(remove_space_before_punctuation(remove_duplicate_punctuation(x))))
# df.head()

In [9]:
re.sub(r"[,:\-–.!;?।]", '', 'তিনি এবং তাঁর মা তাদের: পৈতৃক বাড়িতে থেকে-রত')

'তিনি এবং তাঁর মা তাদের পৈতৃক বাড়িতে থেকেরত'

In [10]:
punctuations= [',', '।', '?', '!', '-', ';', ':']
LABELS= ['O', 'COMMA', 'DARI', 'QUESTION', 'EXCLAMATION', 'HYPHEN', 'SEMICOLON', 'COLON'] 
label2id= {
    'O': 0,
    'COMMA': 1,
    'DARI': 2,
    'QUESTION': 3,
    'EXCLAMATION': 4,
    'HYPHEN': 5,
    'SEMICOLON': 6,
    'COLON':7
}
id2pun= {
     0: ' ',
     1: ',',
     2: '।',
     3: '?',
     4: '!',
     5: '-',
     6: ';',
     7: ':'
}
id2label= {}
for k,v in label2id.items():
    id2label[v]= k

In [11]:
bnorm = Normalizer()
def normalization(sentence):
    _words = [bnorm(word)['normalized']  for word in sentence.split()]
    sentence = " ".join([word for word in _words if word is not None])
    return sentence


def check_pun(text):
    for p in punctuations:
        if p in text:
            return 1
    return 0

def extract_punctuation(text):
    pattern = r'\s*([' + re.escape(''.join(punctuations)) + '])'
    punctuation_matches = re.findall(pattern, text)
    punctuation_list = list(punctuation_matches)
    return punctuation_list

r_pun= []
for i in string.punctuation:
    if i not in punctuations:
        r_pun.append(i)
def remove_unnecessary_punctuation(text):
    # remove unnecessary punctuation with space
    pattern = r'\s*([' + re.escape(''.join(r_pun)) + '])'
    text = re.sub(pattern, '', text)
    return text

def remove_all_punctuation(text):
    re_pun= string.punctuation + '।'
    pattern = r'\s*([' + re.escape(''.join(re_pun)) + '])'
    text= re.sub(pattern, '', text)
    return text

def remove_duplicate_punctuation(text):
    # remove consecutive occurrences of the same punctuation
    pun= string.punctuation + '।'
    pattern = r'\s*([' + re.escape(''.join(pun)) + '])'
    text = re.sub(pattern, r'\1', text)
    return text

def remove_space_before_punctuation(text):
    # remove spaces before punctuation
    pun= string.punctuation + '।'
    pattern = r'\s*([' + re.escape(''.join(pun)) + '])'
    #r'([{}]-)'.format(re.escape(pun))
    text = re.sub(pattern, r'\1', text)
    if text[-1] == ' ':
        text= text[:-1]
    return text

def add_space_after_punctuation(text):
    punctuations= [',', '।', '?', '!', '-', ';', ':'] 
#     puncts= string.punctuation + '।'
    for pun in punctuations:
        text= text.replace(pun, pun + ' ')
    return text

def labeling_data(text):
    punctuations= [',', '।', '?', '!', '-', ';', ':']
    labels= []
    
    for token in text.split():
        label= id2label[0]
        for i, pun in enumerate(punctuations):
            if pun in token:
#                 labels.append(id2label[i+1])
                label= id2label[i+1]
                break
        
        labels.append(label)
#     tokens=re.sub(r"[,:\-–.!;?।]", '', text).split()
    return labels  #, tokens


def prepare_data_for_token_class(df):
    ## take only pun sentence and extract punctuations from sentence
    ## apply some pre-processing for removing ambiguious data
    df['sentence']= df['sentence'].apply(lambda x: normalization(x))
    df['pun']= df['sentence'].apply(lambda x: check_pun(x))
    df= df[df['pun']==1].reset_index(drop= True)
    df['modified_sentence']= df['sentence'].apply(lambda x:  \
                                                  add_space_after_punctuation(remove_space_before_punctuation(remove_duplicate_punctuation(remove_unnecessary_punctuation(x))))
                                                 )


    df['punctuations']= df['sentence'].apply(lambda x: extract_punctuation(x))
    ## extract tokens and generate labels
    re_pun= string.punctuation + '।'
    pattern = r'\s*([' + re.escape(''.join(re_pun)) + '])'
    df['tokens']=df['modified_sentence'].apply(lambda x: remove_all_punctuation(x).split()) # removing punctuation from input tokens #re.sub(pattern, '', x)
    df['labels']= df['modified_sentence'].apply(labeling_data)

    ### sanity check for dataset
    df['t_len']=df['tokens'].apply(len)
    df['l_len']= df['labels'].apply(len)
    df['sanity']= df.apply(lambda x: 1 if x.t_len == x.l_len else 0, axis=1)

    df= df[df['sanity']==1].reset_index(drop= True)

    ## add some auxilary informations
    for k, v in label2id.items():
        df[k]= df['labels'].apply(lambda x: x.count(k))

    for k in label2id.keys():
        print(k, ':' ,len(df[df[k] != 0])) 
    
    return df

In [12]:
def preprocessing(text):
    text= normalization(text)
    text= add_space_after_punctuation(remove_space_before_punctuation(remove_duplicate_punctuation(remove_unnecessary_punctuation(text))))
    text= remove_all_punctuation(text).split()
    return text


In [13]:
display(label2id)
display(id2label)

{'O': 0,
 'COMMA': 1,
 'DARI': 2,
 'QUESTION': 3,
 'EXCLAMATION': 4,
 'HYPHEN': 5,
 'SEMICOLON': 6,
 'COLON': 7}

{0: 'O',
 1: 'COMMA',
 2: 'DARI',
 3: 'QUESTION',
 4: 'EXCLAMATION',
 5: 'HYPHEN',
 6: 'SEMICOLON',
 7: 'COLON'}

1. Every punctuation should attached to its previous word
2. labeling is done for each word not punctuation

In [14]:
# def labeling_data(text):
#     punctuations= [',', '।', '?', '!', '-', ';', ':']
#     labels= []
    
#     for token in text.split():
#         temp= False
#         for i, pun in enumerate(punctuations):
#             if pun in token:
#                 labels.append(id2label[i+1])
#                 temp= True
#                 break
#         if not temp:
#             labels.append(id2label[0])
# #     tokens=re.sub(r"[,:\-–.!;?।]", '', text).split()
#     return labels  #, tokensa


## data preparation

In [15]:
# punctuations= [',', '।', '?', '!', '-', ';', ':']
# label2id= {
#     'O': 0,
#     'COMMA': 1,
#     'DARI': 2,
#     'QUESTION': 3,
#     'EXCLAMATION': 4,
#     'HYPHEN': 5,
#     'SEMICOLON': 6,
#     'COLON':7
# }
# id2label= {}
# for k,v in label2id.items():
#     id2label[v]= k

# def check_pun(text):
#     for p in punctuations:
#         if p in text:
#             return 1
#     return 0

# def extract_punctuation(text):
#     pattern = r'\s*([' + re.escape(''.join(punctuations)) + '])'
#     punctuation_matches = re.findall(pattern, text)
#     punctuation_list = list(punctuation_matches)
#     return punctuation_list

# def remove_duplicate_punctuation(text):
#     # remove consecutive occurrences of the same punctuation
#     pattern = r'([,!?;:।-])\1+'
#     text = re.sub(pattern, r'\1', text)
#     return text
# def remove_space_before_punctuation(text):
#     # remove spaces before punctuation
#     pattern = r'\s*([' + re.escape(''.join(punctuations)) + '])'
#     #r'([{}]-)'.format(re.escape(pun))
#     text = re.sub(pattern, r'\1', text)
#     if text[-1] == ' ':
#         text= text[:-1]
#     return text
# def add_space_after_punctuation(text):
#     punctuations= [',', '।', '?', '!', '-', ';', ':']
#     for pun in punctuations:
#         text= text.replace(pun, pun + ' ')
#     return text

# def labeling_data(text):
#     punctuations= [',', '।', '?', '!', '-', ';', ':']
#     labels= []
    
#     for token in text.split():
#         temp= False
#         for i, pun in enumerate(punctuations):
#             if pun in token:
#                 labels.append(id2label[i+1])
#                 temp= True
#                 break
#         if not temp:
#             labels.append(id2label[0])
# #     tokens=re.sub(r"[,:\-–.!;?।]", '', text).split()
#     return labels  #, tokens

In [16]:
# # read data
# data_path= "/kaggle/input/bengaliai-speech/train.csv"
# df= pd.read_csv(data_path)
# df= df[df['split']=='valid'].reset_index(drop= True)
# # df.head()


# ## take only pun sentence and extract punctuations from sentence
# ## apply some pre-processing for removing ambiguious data
# df['pun']= df['sentence'].apply(lambda x: check_pun(x))
# df['punctuations']= df['sentence'].apply(lambda x: extract_punctuation(x))
# df= df[df['pun']==1].reset_index(drop= True)
# df['modified_sentence']= df['sentence'].apply(lambda x: add_space_after_punctuation(remove_space_before_punctuation(remove_duplicate_punctuation(x))))


# ## extract tokens and generate labels
# df['tokens']=df['modified_sentence'].apply(lambda x: re.sub(r"[,:\-–.!;?।]", '', x).split()) # removing punctuation from input tokens
# df['labels']= df['modified_sentence'].apply(labeling_data)

# ### sanity check for dataset
# df['t_len']=df['tokens'].apply(len)
# df['l_len']= df['labels'].apply(len)
# df['sanity']= df.apply(lambda x: 1 if x.t_len == x.l_len else 0, axis=1)

# df= df[df['sanity']==1].reset_index(drop= True)

# ## add some auxilary informations
# for k, v in label2id.items():
#     df[k]= df['labels'].apply(lambda x: x.count(k))

# for k in label2id.keys():
#     print(k, ':' ,len(df[df[k] != 0])) 

In [17]:
# df1= pd.read_csv("/kaggle/input/punctuation-restoration-dataset-asr/training_data_0.csv")
# df2= pd.read_csv("/kaggle/input/punctuation-restoration-dataset-asr/training_data_1.csv")
# df3= pd.read_csv("/kaggle/input/punctuation-restoration-dataset-asr/training_data_2.csv")
# df4= pd.read_csv("/kaggle/input/punctuation-restoration-dataset-asr/training_data_11.csv")
# df5= pd.read_csv("/kaggle/input/punctuation-restoration-dataset-asr/training_data_12.csv")


1. 12, 11, -> dlsprint
2. 0, 2 -> gec valid, test

In [18]:
# columns= ['sentence', 'tokens', 'labels'] + LABELS
# # df1= df[df[columns]]
# # df1
# columns
# # df1= df[df['sentence','tokens','labels','O','COMMA','DARI','QUESTION','EXCLAMATION','HYPHEN','SEMICOLON','COLON']]
# df1= df1[columns]
# df2= df2[columns]
# df3= df3[columns]
# df4= df4[columns]
# df5= df5[columns]
# len(df1),len(df2), len(df3), len(df4), len(df5)

In [19]:
# for k in label2id.keys():
#     print(k, ':' ,len(df2[df2[k] != 0])) 

In [20]:
# import random
# df0= df2.sample(n= 150_000, random_state= 75).reset_index(drop= True)
# for k in label2id.keys():
#     print(k, ':' ,len(df0[df0[k] != 0])) 

In [21]:
# df= df0.append(df1).append(df3).append(df4).append(df5).reset_index(drop=True)
# df

In [22]:
b_tokenizer= BasicTokenizer()
b_tokenizer("একজন কমিক-ফ্যান { থেকে শুরু'করে চলচ্চিত্র প্রেমি?")

['একজন',
 'কমিক',
 '-',
 'ফ্যান',
 '{',
 'থেকে',
 'শুরু',
 "'",
 'করে',
 'চলচ্চিত্র',
 'প্রেমি',
 '?']

In [23]:
# for k in label2id.keys():
#     print(k, ':' ,len(df[df[k] != 0])) 

In [24]:
# valid_data= pd.read_csv("/kaggle/input/punctuation-restoration-dataset-asr/test_data.csv")
# valid_data= valid_data[columns]
# # valid_df
# for k in label2id.keys():
#     print(k, ':' ,len(valid_data[valid_data[k] != 0])) 

In [25]:
# df['length']= df['sentence'].apply(len)
# display(df)
# df.describe()

### Deleting Duplicate data & recreating data

In [26]:
# sentences= list(df['sentence'].values)
# # (len(sentences))
# display(len(set(sentences)))

# data= pd.DataFrame({'sentence': list(set(sentences))})
# data= prepare_data_for_token_class(data)
# data

In [27]:
# b_sentences= list(valid_data['sentence'].values)
# # (len(sentences))
# display(len(set(b_sentences)))

# valid_data= pd.DataFrame({'sentence': list(set(b_sentences))})
# valid_data= prepare_data_for_token_class(valid_data)
# valid_data

In [28]:
def convert_type(tokens):
    if type(tokens) == str:
        return ast.literal_eval(tokens)
    elif type(tokens)== list:
        return tokens
    else:
        print("Type Error")

# data['tokens']= data['tokens'].apply(lambda x: convert_type(x))
# valid_data['tokens']= valid_data['tokens'].apply(lambda x: convert_type(x))
# data['labels']= data['labels'].apply(lambda x: convert_type(x))
# valid_data['labels']= valid_data['labels'].apply(lambda x: convert_type(x))

In [29]:
# #r"[,:\-–.!;?।]"
# import string
# re_pun= string.punctuation + '।'
# pattern= r'\s*([' + re.escape(''.join(re_pun)) + '])'
# data['sentence']= data['sentence'].apply(lambda x: re.sub(pattern, '', add_space_after_punctuation(remove_space_before_punctuation(remove_duplicate_punctuation(x)))))
# valid_data['sentence']= valid_data['sentence'].apply(lambda x: re.sub(pattern, '', add_space_after_punctuation(remove_space_before_punctuation(remove_duplicate_punctuation(x)))))
# #
# df['tokens']=df['modified_sentence'].apply(lambda x: re.sub(r"[,:\-–.!;?।]", '', x).split())

In [30]:
data= pd.read_parquet("/kaggle/input/bangla-gec/data.parquet")
valid_data= pd.read_parquet("/kaggle/input/bangla-gec/new_valid_cse.parquet")
data

,sentence,pun,modified_sentence,punctuations,tokens,labels,t_len,l_len,sanity,O,COMMA,DARI,QUESTION,EXCLAMATION,HYPHEN,SEMICOLON,COLON
0,"সাংবাদিকরা যখন তাকে আবার জিজ্ঞেস করে , এর অর্থ...",1,"সাংবাদিকরা যখন তাকে আবার জিজ্ঞেস করে, এর অর্থ...","[,, ?]","[সাংবাদিকরা, যখন, তাকে, আবার, জিজ্ঞেস, করে, এর...","[O, O, O, O, O, COMMA, O, O, O, O, O, O, O, O,...",16,16,1,14,1,0,1,0,0,0,0
1,"প্রাথমিক উক্তি যা-ই হোক না কেন , নৌকা বাইচের আ...",1,"প্রাথমিক উক্তি যা- ই হোক না কেন, নৌকা বাইচের ...","[-, ,, ।]","[প্রাথমিক, উক্তি, যা, ই, হোক, না, কেন, নৌকা, ব...","[O, O, HYPHEN, O, O, O, COMMA, O, O, O, O, O, ...",16,16,1,13,1,1,0,0,1,0,0
2,"তিনি বর্ণনা করেন: ""দীর্ঘ সময় ধরে, মানুষের পূর্...",1,"তিনি বর্ণনা করেন: দীর্ঘ সময় ধরে, মানুষের পূর্...","[:, ,, ,, ।]","[তিনি, বর্ণনা, করেন, দীর্ঘ, সময়, ধরে, মানুষের,...","[O, O, COLON, O, O, COMMA, O, O, O, O, O, O, O...",22,22,1,18,2,1,0,0,0,0,1
3,পতৌদি যদি আমি সেটা না করি ?,1,পতৌদি যদি আমি সেটা না করি?,[?],"[পতৌদি, যদি, আমি, সেটা, না, করি]","[O, O, O, O, O, QUESTION]",6,6,1,5,0,0,1,0,0,0,0
4,ঊষর আন্দিজের খনি আর কৃষিজমি সম্বল করে গড়ে ওঠা ...,1,ঊষর আন্দিজের খনি আর কৃষিজমি সম্বল করে গড়ে ওঠা ...,"[,, ,, -, ।]","[ঊষর, আন্দিজের, খনি, আর, কৃষিজমি, সম্বল, করে, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",29,29,1,25,2,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116291,এটা কোনো শিল্পীর আঁকা ছবি নয় বরং ডিজিটাল ক্যাম...,1,এটা কোনো শিল্পীর আঁকা ছবি নয় বরং ডিজিটাল ক্যাম...,"[।, ?]","[এটা, কোনো, শিল্পীর, আঁকা, ছবি, নয়, বরং, ডিজিট...","[O, O, O, O, O, O, O, O, O, O, O, O, O, DARI, ...",18,18,1,16,0,1,1,0,0,0,0
116292,পুলিশ এসে গ্রেফতার করে নিয়ে যায় ক্লেননকে !,1,পুলিশ এসে গ্রেফতার করে নিয়ে যায় ক্লেননকে!,[!],"[পুলিশ, এসে, গ্রেফতার, করে, নিয়ে, যায়, ক্লেননকে]","[O, O, O, O, O, O, EXCLAMATION]",7,7,1,6,0,0,0,1,0,0,0
116293,খেলায় তারা ২-১ গোলে জয়ী হয় ।,1,খেলায় তারা ২- ১ গোলে জয়ী হয়।,"[-, ।]","[খেলায়, তারা, ২, ১, গোলে, জয়ী, হয়]","[O, O, HYPHEN, O, O, O, DARI]",7,7,1,5,0,1,0,0,1,0,0
116294,নাকি রুশ নেতৃত্বের সঙ্গে তার এমন কিছু গুপ্ত সম...,1,নাকি রুশ নেতৃত্বের সঙ্গে তার এমন কিছু গুপ্ত সম...,[?],"[নাকি, রুশ, নেতৃত্বের, সঙ্গে, তার, এমন, কিছু, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, QUE...",15,15,1,14,0,0,1,0,0,0,0


In [31]:
display(len(data))
for k in label2id.keys():
    print(k, ':' ,len(data[data[k] != 0])) 

116296

O : 116281
COMMA : 51561
DARI : 66061
QUESTION : 42097
EXCLAMATION : 9848
HYPHEN : 60142
SEMICOLON : 2267
COLON : 7248


In [32]:
display(len(valid_data))
for k in label2id.keys():
    print(k, ':' ,len(valid_data[valid_data[k] != 0])) 

36360

O : 36357
COMMA : 10786
DARI : 16882
QUESTION : 14358
EXCLAMATION : 5828
HYPHEN : 3109
SEMICOLON : 1499
COLON : 5741


valid data from 'csebuetnlp/BanglaParaphrase'

In [33]:

# from datasets import load_dataset
# ds= load_dataset('csebuetnlp/BanglaParaphrase')


# df= ds['train'].to_pandas()
# # df= df[:100_000]
# df.rename(columns={'source': 'sentence'}, inplace= True,)
# df

# df= prepare_data_for_token_class(df)
# display(len(df))
# for k in label2id.keys():
#     print(k, ':' ,len(df[df[k] != 0])) 





# # df.to_parquet("csenlp_dataset.parquet", index= False)
# df= pd.read_parquet("/kaggle/working/csenlp_dataset.parquet")
# df

# df_new1= df[(df['QUESTION']>0) | (df['EXCLAMATION']>0) | (df['SEMICOLON']>0) | (df['COLON']>0)].reset_index()
# display(len(df_new1))
# for k in label2id.keys():
#     print(k, ':' ,len(df_new1[df_new1[k] != 0])) 

# df_new2= df[(df['DARI']>0) & (df['QUESTION']==0) & (df['EXCLAMATION']==0) & (df['SEMICOLON']==0) & (df['COLON']==0)].reset_index()
# display(len(df_new2))
# for k in label2id.keys():
#     print(k, ':' ,len(df_new2[df_new2[k] != 0]))


# df_new2= df_new2.sample(10_000).reset_index(drop= True)
# # df_new2
# for k in label2id.keys():
#     print(k, ':' ,len(df_new2[df_new2[k] != 0]))

# # ii= set(df_new1['sentence']).intersection(set(df_new2['sentence']))
# # ii
# df_new= pd.concat([df_new1, df_new2], axis= 0).reset_index(drop= True)
# df_new
# for k in label2id.keys():
#     print(k, ':' ,len(df_new[df_new[k] != 0]))

# df_new.to_parquet('new_valid_cse.parquet', index= False)

In [34]:
# data.to_csv('train_data.csv', index= False)
# valid_data.to_csv('valid_data.csv', index= False)

# import pyarrow as pa
# import pyarrow.parquet as pq

# data_table = pa.Table.from_pandas(data, preserve_index=False)
# pq.write_table(data_table, 'data.parquet')

# valid_table = pa.Table.from_pandas(valid_data, preserve_index=False)
# pq.write_table(valid_table, 'valid_data.parquet')

### Modeling

In [35]:
# !pip install transformers

In [36]:
from tqdm.auto import tqdm
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import TensorDataset, DataLoader#, Dataset
from torch.optim import AdamW


import transformers 
from transformers import (
    AutoModel,
    AutoConfig,
    AutoTokenizer,
    BertTokenizer,
    AutoModelForSequenceClassification,
    AutoModelForTokenClassification
)

%env TOKENIZERS_PARALLELISM=true

from sklearn.model_selection import StratifiedKFold


# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

env: TOKENIZERS_PARALLELISM=true


In [37]:
# !nvidia-smi

#### Configs

In [38]:
class CONFIG:
    train= True #False #True
    debug= False # #True
    test= True
    seed= 753
    n_folds= 3
    start_epoch= 18
    num_epochs= 8
    punctuations=  [',', '।', '?', '!', '-', ';', ':'] #[',', '।', '?', '!']
    label_names= ['O', 'COMMA', 'DARI', 'QUESTION', 'EXCLAMATION', 'HYPHEN', 'SEMICOLON', 'COLON'] #  #['O', 'COMMA', 'DARI', 'QUESTION', 'EXCLAMATION']
    num_labels= len(label_names)
    model_name="xlm-roberta-large" #"csebuetnlp/banglabert_large"  #"csebuetnlp/banglabert" # #csebuetnlp/BanglaParaphrase
    max_length= 128
    lstm_size= 512
    pretrained= True
    best_score=  0.955
    weight_path="/kaggle/input/punctuation-restoration-token-classification-2nd/best_model.bin" #"/kaggle/input/punctuation-restoration-token-classification/best_model.bin"
    
#     dataset_no= 3 #1: Dataset-1 only # 2: Dataset-2 only # 3: Combined dataset
    do_normalize= True
#     do_downsampling= True
#     do_upsampling= True
#     upsample_size= 1
    train_batch_size= 12
    valid_batch_size= 16
    num_workers= 2
    device= torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    
    gradient_accumulation_steps= 4
    learning_rate= 1e-5
    weight_decay= 1e-3
    scheduler= "CosineAnnealingWarmRestarts" #"CosineAnnealingWarmRestarts" #"linear"
    T_max= int(len(data)/train_batch_size*num_epochs) + 50
    T_0= 50
    min_lr= 1e-9
    
    eps = 1e-6
    betas= [0.9, 0.999]
    
if CONFIG.debug:
    CONFIG.n_folds= 2
    CONFIG.num_epochs=2
    CONFIG.dataset_size= 300
    

Loading prepossed data for dataloader

In [39]:
# ## checking every data have exact number of labels corresponding to its words 
# b_tokenizer= BasicTokenizer()
# data['len_labels']= data['labels'].apply(lambda x: len(x))
# data['len_words']= data['sentences'].apply(lambda x: len(b_tokenizer.tokenize(x)))

In [40]:
# ## spliting data for train and validation

# skf= StratifiedKFold(n_splits= CONFIG.n_folds, random_state= CONFIG.seed, shuffle= True)

# for fold, (train_index, val_index) in enumerate(skf.split(X= data, y= data['per_tag'])):
#     data.loc[val_index, 'fold']= int(fold)
    
# data['fold']= data['fold'].astype(int)

# data= data[['sentences', 'labels', 'fold']]

In [41]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

Models

In [42]:
# label_names=['O', 'B-PER', 'I-PER']
# id2label= {}
# label2id= {}
# for i, label in enumerate(label_names):
#     id2label[i]= label
#     label2id[label] = i

# display(id2label)
# display(label2id)

In [43]:
# tokenizer
tokenizer = AutoTokenizer.from_pretrained(CONFIG.model_name)

In [44]:
label2id

{'O': 0,
 'COMMA': 1,
 'DARI': 2,
 'QUESTION': 3,
 'EXCLAMATION': 4,
 'HYPHEN': 5,
 'SEMICOLON': 6,
 'COLON': 7}

In [45]:
def token_normalization(tokens):
    _words = [bnorm(word)['normalized']  for word in tokens]
    return _words
token_normalization(data['tokens'][0])

['সাংবাদিকরা',
 'যখন',
 'তাকে',
 'আবার',
 'জিজ্ঞেস',
 'করে',
 'এর',
 'অর্থ',
 'কি',
 'তিনি',
 'প্রয়োজনে',
 'ইয়াসির',
 'আরাফাতের',
 'উপরেও',
 'আক্রমণ',
 'করবেন']

In [46]:
def align_labels_with_tokens(tokens, labels):
    new_labels= []
    word_ids= tokens.word_ids()
    previous_word_id= None
    
    for word_id in word_ids:
#         print(word_id)
        if word_id is None:
            label= -100
        elif word_id != previous_word_id:
            label= label2id[labels[word_id]]
        else:
            label= -100 if word_id is None else label2id[labels[word_id]] # [CLS] has token id= 101. [PAD] token id= 0 ## tokenizer.pad_token_id
        previous_word_id= word_id
        new_labels.append(label)
    
    """-100 is used for all of the speciall token ['[CLS], ['SEP'], [PAD]'], which will help to compute loss for slot filling"""
    
    return new_labels

In [47]:
# label2id[labels[word_id]

In [48]:
normalization(data['sentence'][0]), data['sentence'][0]


('সাংবাদিকরা যখন তাকে আবার জিজ্ঞেস করে , এর অর্থ কি তিনি প্রয়োজনে ইয়াসির আরাফাতের উপরেও আক্রমণ করবেন ?',
 'সাংবাদিকরা যখন তাকে আবার জিজ্ঞেস করে , এর অর্থ কি তিনি প্রয়োজনে ইয়াসির আরাফাতের উপরেও আক্রমণ করবেন ?')

In [49]:
import ast
idx= 1
print(data['sentence'][idx])
# text= normalization(ast.literal_eval(data['tokens'][idx]))
text= token_normalization(data['tokens'][idx])

display(text)
print(type(text))
labels= data['labels'][idx]
print(labels)
tt= tokenizer.tokenize(text, truncation= True, padding=True, max_length=128, add_special_tokens= True, is_split_into_words=True) #, is_split_into_words=True) #tokenizer(text, truncation= True, padding=True, max_length=128, is_split_into_words=True)
tokens= tokenizer(text, truncation= True, padding=True, max_length=128, add_special_tokens= True, is_split_into_words=True) #, is_split_into_words=True)

new_labels= align_labels_with_tokens(tokens, labels)

for t, l in zip(tt, new_labels):
    print(t,':', l)
# new_labels

# tokens.tokens()

প্রাথমিক উক্তি যা-ই হোক না কেন , নৌকা বাইচের আদিবাসী প্রসঙ্গ আমাদের আলোচনায় নিয়ে আসা গুরুত্বপূর্ণ ।


['প্রাথমিক',
 'উক্তি',
 'যা',
 'ই',
 'হোক',
 'না',
 'কেন',
 'নৌকা',
 'বাইচের',
 'আদিবাসী',
 'প্রসঙ্গ',
 'আমাদের',
 'আলোচনায়',
 'নিয়ে',
 'আসা',
 'গুরুত্বপূর্ণ']

<class 'list'>
['O' 'O' 'HYPHEN' 'O' 'O' 'O' 'COMMA' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O'
 'DARI']
<s> : -100
▁প্রাথমিক : 0
▁উ : 0
ক্ত : 0
ি : 0
▁যা : 5
▁ই : 0
▁হোক : 0
▁না : 0
▁কেন : 1
▁নৌকা : 0
▁ : 0
বাই : 0
চ : 0
ের : 0
▁আ : 0
দি : 0
বাসী : 0
▁প্রসঙ্গ : 0
▁আমাদের : 0
▁আলোচনা : 0
য় : 0
▁নিয়ে : 0
▁আসা : 0
▁গুরুত্বপূর্ণ : 2
</s> : -100


In [50]:
tokens= tokenizer(text, truncation= True, padding=True, max_length=128, add_special_tokens= True)
tokens.word_ids()#[1:-1]


[None, 0, None, None, None, None]

In [51]:
def error_finding(text, labels):
    try:
        tokens= tokenizer(text, truncation= True, padding=True, max_length=128, add_special_tokens= True) #is_split_into_words=True
        new_labels= align_labels_with_tokens(tokens, labels)
        return 0
    except:
        print(text)
        return 1
    

In [52]:
# data['word_ids']= data['sentence'].apply(lambda x: tokenizer(x, truncation= True, padding=True, max_length=128, add_special_tokens= True).word_ids())
# data['error_finding']= data.apply(lambda x: error_finding(x['modified_sentence'], x['labels']), axis=1)


# sum(data['error_finding'])

In [53]:
# valid_data['error_finding']= valid_data.apply(lambda x: error_finding(x['modified_sentence'], x['labels']), axis=1)
# sum(valid_data['error_finding'])

# sum(valid_data['error_finding'].values)


In [54]:
# data['w_len']= data['word_ids'].apply(len)

# for w, l in tqdm(zip(list(data['word_ids'].values), list(data['labels'].values))):
#     print(len(w), len(l))



In [55]:
from torch.utils.data import DataLoader, Dataset

class CustomDataset(Dataset):
    def __init__(self, df, tokenizer, cfg):
        self.df= df
        self.cfg= cfg
        self.tokenizer= tokenizer
        self.max_length= self.cfg.max_length
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        text= list(self.df.tokens[index]) #normalization(self.df.sentence[index])
        labels= list(self.df.labels[index])
        inputs= self.tokenizer(text, 
                               truncation= True, 
                               padding=True, 
                               max_length=self.max_length,
                               is_split_into_words=True
                               )

        new_labels= align_labels_with_tokens(inputs, labels)
        
        return {
            "input_ids": inputs['input_ids'],
            'attention_mask': inputs['attention_mask'],
            'targets': new_labels            
        }
    

In [56]:
# tokenizer.decode([1])

In [57]:
dataset= CustomDataset(data, tokenizer, CONFIG)
for d in dataset:
    print(d)
    print(len(d['input_ids']), len(d['targets']))
    break

{'input_ids': [0, 111483, 5534, 138061, 115200, 92806, 49775, 155656, 98394, 5507, 11785, 66468, 14403, 25141, 128038, 956, 10814, 16576, 137609, 4979, 5534, 64785, 86260, 72003, 108645, 4979, 171462, 13880, 84016, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'targets': [-100, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, -100]}
30 30


In [58]:
class Collate:
    def __init__(self, tokenizer):
        self.tokenizer= tokenizer
    
    def __call__(self, batch):
        output= dict()
        output["input_ids"] = [sample['input_ids'] for sample in batch]
        output["attention_mask"] = [sample["attention_mask"] for sample in batch]
        output['targets'] = [sample['targets'] for sample in batch]
        
        batch_max= max([len(ids) for ids in output['input_ids']])
        
        # dynamic padding
        if self.tokenizer.padding_side == "right":
            output["input_ids"] = [ids + (batch_max - len(ids))*[self.tokenizer.pad_token_id] for ids in output['input_ids']]
            output['attention_mask']= [mask + (batch_max - len(mask))*[0] for mask in output['attention_mask']]
            output['targets']= [target + (batch_max - len(target))*[-100] for target in output['targets']]
        else:
            output["input_ids"] = [(batch_max - len(ids))*[self.tokenizer.pad_token_id] + ids for ids in output['input_ids']]
            output['attention_mask']= [(batch_max - len(mask))*[0] + mask for mask in output['attention_mask']]
            output['targets']= [(batch_max - len(target))*[-100] + target for target in output['targets']]
        
        # convert array to tensor
        output["input_ids"] = torch.tensor(output['input_ids'], dtype= torch.long)
        output["attention_mask"] = torch.tensor(output['attention_mask'], dtype= torch.long)
        output['targets'] = torch.tensor(output['targets'], dtype=torch.long)#
        
        return output
    

collate_fn= Collate(tokenizer)

In [59]:
# from transformers import DataCollatorForTokenClassification
# collate_fn= DataCollatorForTokenClassification(tokenizer= tokenizer)

In [60]:
def prepare_loader(df_train, df_valid, tokenizer, cfg):
#     df_train= df[df.fold != fold].reset_index(drop= True) # 2 fold out of 3 fold is used as training data, and 1 fold for validation.
#     df_valid= df[df.fold == fold].reset_index(drop= True)
    valid_labels = df_valid['labels'].values
    
    # converting dataFrame to dataset.
    train_dataset= CustomDataset(df_train, tokenizer, cfg)
    valid_dataset= CustomDataset(df_valid, tokenizer, cfg)
    
    train_loader= DataLoader(train_dataset, 
                             batch_size= cfg.train_batch_size, 
                             collate_fn= collate_fn, #merges a list of samples to form a mini-batch of Tensors
                             num_workers= cfg.num_workers, # how many subprocesses to use for data loading
                             shuffle= True, 
                             pin_memory= True,
                             drop_last= False, 
                            )
    
    valid_loader= DataLoader(valid_dataset, 
                            batch_size= cfg.valid_batch_size,
                            collate_fn= collate_fn, 
                            num_workers= cfg.num_workers,
                            shuffle= False,
                            pin_memory= True, #If True, the data loader will copy Tensors into device/CUDA pinned memory before returning them
                            drop_last= False,
                            )
    
    return train_loader, valid_loader

In [61]:
train_loader, valid_loader= prepare_loader(data, valid_data, tokenizer, CONFIG)

In [62]:
i =0
for batch in train_loader:
    print(batch)
    i +=1
    if i== 2:
        break
    

{'input_ids': tensor([[     0,  25141,  24090,  10066,   8377,  64248,      6, 166762,   7277,
          91975,    896,  58041,  20275,   2937,  38254,  36539,  13880, 235527,
           9755, 199003, 118425,  95719,  74046,  53155, 155656,    896,  72237,
         215599, 175310,  25141,   4876,   7277,  91975,    896,  87147,      2,
              1,      1,      1,      1,      1],
        [     0,  27371,  31111,  50442,   2937, 114731, 114731, 160574,   2763,
         224306,  20404,   4577,  30234,   5507,  16525, 114731, 114731,  58973,
          21431,  19796,  56327,  30234,   5507,      2,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1],
        [     0,  10814, 187717, 119977,  73643,   8099,  96772, 111542,   5780,
          86387,      2,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,    

In [63]:
# from torch.nn.utils.rnn import pack_padded_sequence
class NER_MODEL(nn.Module):
    def __init__(self, model_name= None, cfg= CONFIG):
        super(NER_MODEL, self).__init__()
        self.cfg= cfg
        self.num_labels= self.cfg.num_labels
        if model_name != None:
            self.model_name= model_name
        else:
            self.model_name= self.cfg.model_name
            
        self.model_config= AutoConfig.from_pretrained(self.model_name, output_hidden_states= True)
        self.model= AutoModel.from_pretrained(self.model_name, config= self.model_config)
#         self.embeddings= model.embeddings
        
        self.dropout= nn.Dropout(p= 0.2)
        self.lstm = nn.LSTM(self.model_config.hidden_size, self.cfg.lstm_size, batch_first=True, bidirectional=True)

#         self.linear= nn.Linear(self.model_config.hidden_size*2, self.num_labels)
#         self.linear= nn.Linear(self.model_config.hidden_size, self.num_labels)
        self.linear= nn.Linear(self.cfg.lstm_size*2, self.num_labels)
        
    
    def forward(self, input_ids, attention_mask, targets= None):
        
        outputs= self.model(input_ids,
                            attention_mask= attention_mask,
                            )
#         embeddings= self.model.embeddings(input_ids)
        sequence_output= outputs['last_hidden_state'] # sequence_output has the following shape: (batch_size, sequence_length, 768)
          
        lstm_output, (last_hidden, last_cell) = self.lstm(sequence_output) ## extract the 1st token's embeddings
#         hidden = torch.cat((last_hidden[0], last_hidden[1]), dim=1)
        lstm_output= self.dropout(lstm_output)
        entity_logits = self.linear(lstm_output) ### assuming that you are only using the output of the last LSTM cell to perform classification

#         out= torch.cat((sequence_output, embeddings), dim=-1)
#         entity_logits= self.dropout(entity_logits)
#         entity_logits= sequence_output #self.dropout(sequence_output)
#         entity_logits= self.linear(entity_logits)
        
        return entity_logits #lstm_output


In [64]:
model= NER_MODEL(CONFIG.model_name)

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [65]:
# for batch in train_loader:

batch= next(iter(train_loader))
outputs= model(**batch)
#     inp= batch
#     print(outputs)
#     break
    

loss_fn= nn.CrossEntropyLoss(ignore_index= -100)
num_labels= CONFIG.num_labels

# output= outputs[0]
outputs.shape

loss= loss_fn(outputs.view(-1, num_labels),batch['targets'].view(-1))
loss

tensor(2.0905, grad_fn=<NllLossBackward0>)

In [66]:
print(outputs.shape)

# print(outputs[0].shape)
# x=torch.cat((outputs[0], outputs[1]), dim=1)
# x.shape


torch.Size([12, 67, 8])


In [67]:
# np.argmax(output.detach().numpy(), axis= -1)
# batch['targets'][0]

In [68]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=cc111956c252c89fcdfe3e535d710dc8497804b28728fa03d17d1c585147e6ba
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [69]:
# from datasets import load_metric
# metric = load_metric("seqeval")

# def compute_metrics(logits, labels):
#     predictions = np.argmax(logits.detach().cpu().numpy(), axis=-1)

#     # Remove ignored index (special tokens) and convert to labels
#     true_labels = [[ label_names[l] for l in label if l != -100] for label in labels]
#     true_predictions = [
#         [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
#         for prediction, label in zip(predictions, labels)
#     ]
#     all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
#     return all_metrics['overall_f1']
# {
#         "precision": all_metrics["overall_precision"],
#         "recall": all_metrics["overall_recall"],
#         "f1": all_metrics["overall_f1"],
#         "accuracy": all_metrics["overall_accuracy"],
#     }

In [70]:
# len(np.argmax(outputs.detach().numpy(), axis=-1)[0])#, batch['targets']
# true_labels = [[ label_names[l] for l in label if l != -100] for label in batch['targets']]
# len(true_labels[0])

In [71]:
# compute_metrics(outputs, batch['targets'])


In [72]:
# help(metric.compute)

In [73]:
# from datasets import load_metric
# metric = load_metric("seqeval")

# def compute_metrics(logits, labels):
#     predictions = np.argmax(logits.detach().cpu().numpy(), axis=-1)

#     # Remove ignored index (special tokens) and convert to labels
#     true_labels = [[CONFIG.label_names[l] for l in label if l != -100] for label in labels]
#     true_predictions = [
#         [CONFIG.label_names[p] for (p, l) in zip(prediction, label) if l != -100]
#         for prediction, label in zip(predictions, labels)
#     ]
#     all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
#     return all_metrics['overall_f1'], all_metrics['overall_accuracy']

from sklearn.metrics import f1_score
def compute_metrics(logits, labels):
    predictions = np.argmax(logits.detach().cpu().numpy(), axis=-1)
    labels= labels.detach().cpu().numpy()
#     predictions= logits
    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[l for l in label if l != -100] for label in labels]
    true_predictions = [[p for (p, l) in zip(prediction, label) if l != -100]
                        for prediction, label in zip(predictions, labels)]
    f1_scores= []
    accuracy= []
    for truths, preds in zip(true_labels, true_predictions):
        score = f1_score(np.array(truths), np.array(preds), average='weighted')
        correct_preds = np.array(truths) == np.array(preds)
        accuracy.append(np.mean(correct_preds))
        f1_scores.append(score)
    
    return np.mean(f1_scores) , np.mean(accuracy)


def token_loss_fn(logits, labels, attention_mask= None):
    loss_fn= nn.CrossEntropyLoss(ignore_index= -100) # for ignoring special tokens
    num_labels= CONFIG.num_labels
    
    if attention_mask is not None:
        mask= attention_mask.view(-1) == 1 #mask for keeping the effective part
        active_logits= logits.view(-1, num_labels)[mask]
        active_labels= labels.view(-1)[mask]
#         print(active_logits.size(), active_labels.size())
        entity_loss= loss_fn(active_logits, active_labels)
        
    else:
        entity_loss= loss_fn(logits.view(-1, num_labels), labels.view(-1))
    
    return entity_loss

In [74]:
def fetch_scheduler(optimizer):
    if CONFIG.scheduler == "CosineAnnealingLR":
        scheduler= lr_scheduler.CosineAnnealingLR(optimizer, T_max= CONFIG.T_max, eta_min= CONFIG.min_lr)
    elif CONFIG.scheduler == "CosineAnnealingWarmRestarts":
        scheduler= lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0= CONFIG.T_0, eta_min= CONFIG.min_lr)
    elif CONFIG.scheduler== "linear":
        scheduler= lr_scheduler.LinearLR(optimizer, start_factor= 0.01, end_factor= 1.0, total_iters= 100)
    elif CONFIG.scheduler == None:
        return None

    return scheduler

In [75]:
def train_one_epoch(model, dataloader, optimizer, scheduler, epoch, device= CONFIG.device):
    model.train()

    dataset_size= 0
    running_loss= 0.0
    score= []
    accuracy= []
    
    progress_bar= tqdm(enumerate(dataloader), total= len(dataloader))
    steps= len(dataloader)
    for step, data in progress_bar:
        ids= data['input_ids'].to(device, dtype= torch.long)
        masks= data['attention_mask'].to(device, dtype= torch.long)
        targets= data['targets'].to(device, dtype= torch.long)
        
        batch_size= ids.size(0)
        outputs= model(ids, masks)
        loss= token_loss_fn(outputs, targets, attention_mask= masks)
        f1_score, acc= compute_metrics(logits= outputs, labels= targets)
        score.append(f1_score)
        accuracy.append(acc)
        if CONFIG.gradient_accumulation_steps > 1:
            loss= loss/ CONFIG.gradient_accumulation_steps
        
        loss.backward()
        ## Gradient Accumulation
        if (step + 1) % CONFIG.gradient_accumulation_steps == 0 or step == steps:
            
            optimizer.step() #Performs a single optimization step (parameter update)
            
            # clear out the gradients of all Variables 
            # in this optimizer (i.e. W, b)
            optimizer.zero_grad()
            
            if scheduler is not None:
                scheduler.step()
        
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        epoch_loss= running_loss/ dataset_size
        epoch_f1_score= np.mean(score)
        epoch_accuracy= np.mean(accuracy)
        
        progress_bar.set_postfix(Epoch= epoch,
                                 Train_loss= epoch_loss,
                                 F1_Score= epoch_f1_score,
                                 Train_accuracy= epoch_accuracy,
                                 LR= optimizer.param_groups[0]['lr'])
    
    return epoch_loss, epoch_f1_score, epoch_accuracy #

In [76]:
def valid_one_epoch(model, dataloader, epoch, device= CONFIG.device):
    model.eval()
    
    dataset_size= 0
    running_loss= 0.0
    score= []
    accuracy= []
    
    progress_bar= tqdm(enumerate(dataloader), total= len(dataloader))
    steps= len(dataloader)
    
    for step, data in progress_bar:
        ids= data['input_ids'].to(device, dtype= torch.long)
        masks= data['attention_mask'].to(device, dtype= torch.long)
        targets= data['targets'].to(device, dtype= torch.long)
        
        batch_size= ids.size(0)
        
        with torch.no_grad():
            outputs= model(ids, masks)
            loss= token_loss_fn(outputs, targets, attention_mask= masks)
            f1_score, acc= compute_metrics(logits= outputs, labels= targets)
        
        score.append(f1_score)
        accuracy.append(acc)
        if CONFIG.gradient_accumulation_steps > 1:
            loss= loss/ CONFIG.gradient_accumulation_steps
        
        
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        epoch_loss= running_loss/ dataset_size
        epoch_f1_score= np.mean(score)
        epoch_accuracy= np.mean(accuracy)
        
        progress_bar.set_postfix(Epoch= epoch,
                                 Valid_loss= epoch_loss,
                                 Valid_F1_Score= epoch_f1_score,
                                 Valid_accuracy= epoch_accuracy
                                )
    return epoch_loss, epoch_f1_score, epoch_accuracy  #

In [77]:
def add_punctuation(tokens, predictions, word_ids):
    result= ''
    for i, d in enumerate(zip(word_ids, predictions)):
        word_id, pred= d
        if i== 0: # for first token
            previous_word_id= word_id
            result += tokens[word_id] + id2pun[pred] 
        else:
            if word_id== previous_word_id: # if word id matches [label is for same token, so don't do anything]
                pass
            else:
                previous_word_id= word_id
                result +=' ' +tokens[word_id] + id2pun[pred] 
    if len(result)>0:
        result= normalization(result)
    else:
        result= '।'
    return result   

def run_sample_test(epoch, model, tokenizer, config= CONFIG):
    print(f"############ Running sample test on epoch {epoch} ############")
    test_df= pd.read_csv('/kaggle/input/bangla-gec/test_data.csv')
    random.seed(config.seed)
    sentences= random.sample(list(test_df['gt_sentence'].values), k= 5)
    for text in sentences:
        orig_sent= text
        text= preprocessing(text)
    #     tokens= toenizer.
        inputs= tokenizer(text, padding=True, truncation=True, is_split_into_words= True, return_tensors="pt")
        word_ids= inputs.word_ids()[1:-1]
        outputs= model(inputs['input_ids'].to(config.device), inputs['attention_mask'].to(config.device))
        outputs= outputs.detach().cpu().numpy().argmax(axis= -1)[0, 1:-1]
        result= add_punctuation(text, outputs, word_ids)
        print(f'Input sentence:     {orig_sent}')
        print(f'Predicted sentence: {result}')
        

In [78]:
import time
from collections import defaultdict
def training_loop(model, tokenizer, optimizer, scheduler, num_epochs= CONFIG.num_epochs, patience= 3):
    
    start= time.time()
    best_loss= np.inf
    if CONFIG.pretrained:
        best_score= CONFIG.best_score
    else:
        best_score= 0
    trigger_times= 0
    history= defaultdict(list)
    
    for epoch in range(CONFIG.start_epoch, CONFIG.start_epoch + num_epochs + 1):
        t1= time.time()
        train_epoch_loss, train_f1_score, train_accuracy= train_one_epoch(model, train_loader, optimizer, scheduler, epoch, CONFIG.device)
        valid_epoch_loss, valid_f1_score, valid_accuracy = valid_one_epoch(model, valid_loader, epoch, CONFIG.device)
        
        history['train_loss'].append(train_epoch_loss)
        history['valid_loss'].append(valid_epoch_loss)
        history['train_f1_score'].append(train_f1_score)
        history['valid_f1_score'].append(valid_f1_score)

        LOGGER.info(f'Epoch {epoch+1} - avg_train_loss: {train_epoch_loss:.4f}  avg_val_loss: {valid_epoch_loss:.4f}  time: {time.time() - t1:.0f}s')
        LOGGER.info(f'Epoch {epoch+1} - Train F1 Score: {train_f1_score:.4f} - Train Accuracy: {train_accuracy:.4f} - Valid F1 Score: {valid_f1_score:.4f} - Valid Accuracy: {valid_accuracy:.4f}')
        #####
        if  valid_accuracy >= best_score: #valid_epoch_loss #best_loss #
            trigger_times= 0
            print(f"Vlaidation Accuracy Improved {best_score} ---> {valid_accuracy}") #valid_f1_score
            best_score= valid_accuracy
            
            path= f"best_model.bin"
            torch.save(model.state_dict(), path)
            print(f"Model saved to {path}")
            run_sample_test(epoch, model, tokenizer, config= CONFIG)
        else:
            trigger_times += 1
            if trigger_times >= patience:
                print("Early Stoping. \n")
                break
                
    time_elapsed= time.time() - start
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    
    return history, valid_epoch_loss, best_score



In [79]:
# del train_loader, valid_loader
# !nvidia-smi
# torch.cuda.empty_cache()
# gc.collect()

In [80]:
if CONFIG.train:
    if CONFIG.debug:
        data= data[:100].reset_index(drop= True)
        valid_data= data[:50].reset_index(drop= True)
#     fold_scores= []
#     for fold in range(CONFIG.n_folds):


    torch.cuda.empty_cache()
#         print(f"====== Fold: {fold} ======")

    # Create Dataloaders
    train_loader, valid_loader = prepare_loader(data, valid_data, tokenizer, cfg= CONFIG)

    # Model
    model= NER_MODEL(cfg= CONFIG)
    if CONFIG.pretrained:
        model.load_state_dict(torch.load(CONFIG.weight_path, map_location= CONFIG.device))
    model.to(device= CONFIG.device)
    # Optimizer & LR Scheduler
    optimizer= AdamW(model.parameters(), lr= CONFIG.learning_rate, weight_decay= CONFIG.weight_decay, eps= CONFIG.eps, betas= CONFIG.betas)
    scheduler= fetch_scheduler(optimizer)

    # run training
    history, loss, f1_score= training_loop(model, tokenizer, optimizer, scheduler, CONFIG.num_epochs, patience= 5)#epoch_loss

    print("\n\n")
#         print(f"Fold [{fold}] avg loss: {epoch_loss}\n")
#         print(f"Fold [{fold}] avg score: {f1_score}\n")
#         fold_scores.append(f1_score)

#         if fold < CONFIG.n_folds-1:
#             del model
    del train_loader, valid_loader

    print(f"====== xxxx ======")
    print(f"Overall F1 scores: {f1_score}")
    # print(f"Overall average scores: {np.mean(fold_scores, axis= 0)}")
#     print(f"Overall score: {np.mean(np.mean(fold_scores, axis= 0))}")
    print(f"====== xxxx ======")


Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/9692 [00:00<?, ?it/s]

  0%|          | 0/2273 [00:00<?, ?it/s]

Epoch 19 - avg_train_loss: 0.0050  avg_val_loss: 0.0523  time: 3595s
Epoch 19 - Train F1 Score: 0.9915 - Train Accuracy: 0.9917 - Valid F1 Score: 0.9600 - Valid Accuracy: 0.9571


Vlaidation Accuracy Improved 0.955 ---> 0.9570863462857453
Model saved to best_model.bin
############ Running sample test on epoch 18 ############
Input sentence:     বিশেষ করে, ডেনিমের নীল জিন্সগুলো ছিল হিপ্পি সম্প্রদায়ের সাধারণ পোশাক।
Predicted sentence: বিশেষ করে, ডেনিমের নীল জিন্সগুলো ছিল হিপ্পি সম্প্রদায়ের সাধারণ পোশাক!
Input sentence:     জয়ের জন্য শেষ ওভারে নিউজিল্যান্ডের প্রয়োজন ছিলো ১৫ রান।
Predicted sentence: জয়ের জন্য শেষ ওভারে নিউজিল্যান্ডের প্রয়োজন ছিলো ১৫ রান!
Input sentence:     অমুসলিমদের যেহেতু ইসলামের নিয়ম-কানুন মানতে হয় না, তাই এই ধরণের অভিযানে কর্তৃপক্ষ অমুসলিমদের লক্ষ্যবস্তু করে না বলেও জানান মি. ইয়াহইয়া।
Predicted sentence: অমুসলিমদের যেহেতু ইসলামের নিয়ম- কানুন মানতে হয় না, তাই এই ধরণের অভিযানে কর্তৃপক্ষ অমুসলিমদের লক্ষ্যবস্তু করে না বলেও জানান মি ইয়াহইয়া।
Input sentence:     এরপর গত তিনদিনে আরো ২৭ জনের মৃত্যু হয়েছে।
Predicted sentence: এরপর গত তিনদিনে আরো ২৭ জনের মৃত্যু হয়েছে!
Input sentence:     ধাঁধাটি ছিলো এরকম-'প্রথমে একজন ছদ্মবেশে থাকা মানুষের কথা চিন্তা করো

...

Epoch 26 - avg_train_loss: 0.0032  avg_val_loss: 0.0538  time: 3589s
Epoch 26 - Train F1 Score: 0.9942 - Train Accuracy: 0.9943 - Valid F1 Score: 0.9608 - Valid Accuracy: 0.9582


  0%|          | 0/9692 [00:00<?, ?it/s]

  0%|          | 0/2273 [00:00<?, ?it/s]

Epoch 27 - avg_train_loss: 0.0031  avg_val_loss: 0.0534  time: 3596s
Epoch 27 - Train F1 Score: 0.9945 - Train Accuracy: 0.9944 - Valid F1 Score: 0.9624 - Valid Accuracy: 0.9590


Early Stoping. 

Training complete in 8h 59m 56s



====== xxxx ======
Overall F1 scores: 0.9603746411701592
====== xxxx ======


## Inference

In [81]:
# model_checkpoint= "/kaggle/input/hishab-ner/best_model_0.bin" #"/kaggle/working/best_model_0.bin"
# model_name= "nafi-zaman/celloscope-28000-ner-banglabert-finetuned"
# tokenizer= AutoTokenizer.from_pretrained(model_name)
# model= NER_MODEL(model_name= model_name, cfg= CONFIG)
# model.load_state_dict(torch.load(model_checkpoint, map_location=  CONFIG.device))
# model.to(CONFIG.device)

if CONFIG.test and (CONFIG.train == False):
    weight_path= "/kaggle/input/punctuation-restoration-token-classification/best_model.bin"
    model=NER_MODEL(model_name= CONFIG.model_name)
    model.load_state_dict(torch.load(weight_path, map_location= CONFIG.device))
    model.to(CONFIG.device)

model.eval()

NER_MODEL(
  (model): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
    

In [82]:
def add_punctuation(tokens, predictions, word_ids):
    result= ''
    for i, d in enumerate(zip(word_ids, predictions)):
        word_id, pred= d
        if i== 0: # for first token
            previous_word_id= word_id
            result += tokens[word_id] + id2pun[pred] 
        else:
            if word_id== previous_word_id: # if word id matches [label is for same token, so don't do anything]
                pass
            else:
                previous_word_id= word_id
                result += tokens[word_id] + id2pun[pred] 
    
    result= normalization(result)
    return result        
# #         if pred == 0:
#         result += tok + id2pun[pred] 
#     return result
#         else:
#             result += ' ' + tok+id2label[]

def pun_inference_fn(text, model, tokenizer, config= CONFIG):
    orig_sent= text
    text= preprocessing(text)
#     tokens= toenizer.
    inputs= tokenizer(text, padding=True, truncation=True, is_split_into_words= True, return_tensors="pt")
    word_ids= inputs.word_ids()[1:-1]
    outputs= model(inputs['input_ids'].to(config.device), inputs['attention_mask'].to(config.device))
    outputs= outputs.detach().cpu().numpy().argmax(axis= -1)[0, 1:-1]
    result= add_punctuation(text, outputs, word_ids)
    print(f'Input sentence:     {orig_sent}')
    print(f'Predicted sentence: {result}')
#     print(orig_sent, outputs, result)
    return result
#     for token, out in zip(tokenizer.tokenize(text), outputs):
#         print(f"{token:>10} : {id2label[out]}")
    

In [83]:
# text= dataset1_test['sentences'][550]
# print(text)



# inputs= tokenizer(text, padding=True, truncation=True, return_tensors="pt")
# # inputs= 
# outputs= model(inputs['input_ids'].to(CONFIG.device), inputs['attention_mask'].to(CONFIG.device))
# outputs= outputs.detach().cpu().numpy().argmax(axis= -1)[0, 1:-1]

# for token, out in zip(tokenizer.tokenize(text), outputs):
#     print(f"{token:>10} : {id2label[out]}")

In [84]:
text= valid_data['sentence'][1]

print(text)
print(preprocessing(text))

result= pun_inference_fn(text, model, tokenizer, CONFIG)

কুনিমুন্ডের খুলিকে নিজের মদ্যপানের পাত্র হিসেবে ব্যবহার করতেন আল্বোইন!
['কুনিমুন্ডের', 'খুলিকে', 'নিজের', 'মদ্যপানের', 'পাত্র', 'হিসেবে', 'ব্যবহার', 'করতেন', 'আল্বোইন']
Input sentence:     কুনিমুন্ডের খুলিকে নিজের মদ্যপানের পাত্র হিসেবে ব্যবহার করতেন আল্বোইন!
Predicted sentence: কুনিমুন্ডের খুলিকে নিজের মদ্যপানের পাত্র হিসেবে ব্যবহার করতেন আল্বোইন!


In [85]:
test_df= pd.read_csv('/kaggle/input/bangla-gec/test_data.csv')
# test_df
sentences= list(test_df['gt_sentence'].values)[:50]

In [86]:
sentences= list(test_df['gt_sentence'].values)[:50]
for sentence in sentences:
    result= pun_inference_fn(sentence, model, tokenizer, CONFIG)
    

Input sentence:     এই প্রক্রিয়ায় কোরীয় সরকার প্রায় ৯, ০০০ জনকে টেস্ট করল, যারা করোনাভাইরাসে আক্রান্ত কারো সংস্পর্শে এসেছিলেন।
Predicted sentence: এই প্রক্রিয়ায় কোরীয় সরকার প্রায় ৯,০০০ জনকে টেস্ট করল-যারা করোনাভাইরাসে আক্রান্ত কারো সংস্পর্শে এসেছিলেন।
Input sentence:     চন্দ্র-পানি-লতা প্যাটার্নের সব থেকে সফল নজির দেখা যায় স্বর্গীয় পানীয় ভারতীয় সোম এবং পারসিক হাওমাতে।
Predicted sentence: চন্দ্র-পানি-লতা প্যাটার্নের সব থেকে সফল নজির দেখা যায় স্বর্গীয় পানীয় ভারতীয় সোম এবং পারসিক হাওমাতে।
Input sentence:     আগ্রহের বিষয় হল, সেই একই বছরে চার্লস, সারা ম্যাককরকুডেলের প্রেমিক হিসেবে ডায়ানার সঙ্গে প্রথম দেখা করেন, যার সঙ্গে তার পরিচয় হয়েছিল।
Predicted sentence: আগ্রহের বিষয় হল,সেই একই বছরে চার্লস সারা ম্যাককরকুডেলের প্রেমিক হিসেবে ডায়ানার সঙ্গে প্রথম দেখা করেন,যার সঙ্গে তার পরিচয় হয়েছিল।
Input sentence:     মাটি থেকে কোন কিছু গড়ে তোলার চেয়ে ভেঙে যাওয়া কিছু জোড়া লাগানো ঢের কঠিন, আর এই কঠিন কাজটাই জিদানকে এবার করতে হবে।
Predicted sentence: মাটি থেকে কোন কিছু গড়ে তোলার চেয়ে ভেঙে যাওয়া কিছু জোড়া

## Random

In [87]:
def demo_inference_fn(text, label, model, tokenizer, config= CONFIG):
    orig_sent= text
    text= preprocessing(text)
#     tokens= toenizer.
    inputs= tokenizer(text, padding=True, truncation=True, max_length= 128, is_split_into_words= True, return_tensors="pt")
    labels= align_labels_with_tokens(inputs, label)
#     word_ids= inputs.word_ids()[1:-1]
    outputs= model(inputs['input_ids'].to(config.device), inputs['attention_mask'].to(config.device))
    outputs= outputs.detach().cpu().numpy().argmax(axis= -1) #[0, 1:-1]
# #     result= add_punctuation(text, outputs, word_ids)
#     print(f'Input sentence:     {orig_sent}')
#     print(f'Predicted sentence: {result}')

    return labels, outputs



In [88]:
# idxs= [i for i in dataset1_train[dataset1_train['per_tag']==1].index][:10]
text= data['sentence'][1]
label= data['labels'][1]
# print(text, labels)
# print(text)
# print(preprocessing(text))

result= demo_inference_fn(text, label ,model, tokenizer, CONFIG)
lab, out= result

In [89]:
lab, out[0]
labels = [lab]


In [90]:
true_labels= np.array([[l for l in label if l != -100] for label in labels])
type(true_labels)

numpy.ndarray

In [91]:
true_predictions = [
    [p for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(out, labels)
]
true_predictions

[[0, 0, 0, 0, 5, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2]]

In [92]:
from sklearn.metrics import f1_score
def demo_compute_metrics(logits, labels):
#     predictions = np.argmax(logits.detach().cpu().numpy(), axis=-1)
    predictions= logits
    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[l for l in label if l != -100] for label in labels]
    true_predictions = [[p for (p, l) in zip(prediction, label) if l != -100]
                        for prediction, label in zip(predictions, labels)]
    f1_scores= []
    accuracy= []
    for lab, pred in zip(true_labels, true_predictions):
        score = f1_score(np.array(lab), np.array(pred), average='macro')
        correct_pred = np.array(lab) == np.array(pred)
        accuracy.append(np.mean(correct_pred))
        f1_scores.append(score)
    
    return np.mean(f1_scores) , np.mean(accuracy)


In [93]:
truth = [
    [0, 0, 5, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2],
    [1, 2, 0, 3]
]
predictions = [
    [0, 0, 5, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2],
    [0, 2, 0, 3]
]

demo_compute_metrics(predictions, truth)

(0.787037037037037, 0.8455882352941176)

In [94]:
# # for text in dataset1_train['sentences'][idxs]:
# #     inference_fn(text, model, tokenizer, CONFIG)
# from sklearn.preprocessing import MultiLabelBinarizer

# truth = np.array([
#     [0, 0, 5, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2],
#     [1, 2, 0, 3]
# ])
# predictions = np.array([
#     [0, 0, 5, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2],
#     [0, 2, 0, 3]
# ])
# demo_compute_metrics(predictions, truth)
# # mlb = MultiLabelBinarizer()
# # truth_binary = mlb.fit_transform(truth)
# # predictions_binary = mlb.transform(predictions)
# # truth_binary,predictions_binary

In [95]:
import numpy as np
tp = np.zeros(1+len(id2label), dtype=np.int)
tp

array([0, 0, 0, 0, 0, 0, 0, 0, 0])

In [96]:
from sklearn.metrics import f1_score
import numpy as np

truth = [
    [0, 0, 5, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2],
    [1, 2, 0, 3]
]
predictions = [
    [0, 0, 5, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2],
    [0, 2, 0, 3]
]

# Initialize a list to store F1 scores for each sample
f1_scores = []

# Calculate F1 score for each sample
for true_labels, pred_labels in zip(truth, predictions):
    f1 = f1_score(true_labels, pred_labels, average='macro')
    f1_scores.append(f1)

# Calculate the average F1 score across all samples
average_f1_score = np.mean(f1_scores)

# Print individual F1 scores and the average F1 score
for i, f1 in enumerate(f1_scores):
    print(f"F1 score for sample {i + 1}: {f1}")
print(f"Average F1 score: {average_f1_score}")


F1 score for sample 1: 0.9074074074074074
F1 score for sample 2: 0.6666666666666666
Average F1 score: 0.787037037037037


In [97]:
ner_model= NER_MODEL(CONFIG.model_name)
ner_model.eval()

model= AutoModel.from_pretrained(CONFIG.model_name)
model.eval()

# model.encoder
model_config= AutoConfig.from_pretrained(CONFIG.model_name, output_hidden_states= True)
model_config#.hidden_size

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a

XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-large",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "output_hidden_states": true,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.30.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

In [98]:
# display(model.ElectraEmbeddings)
# from transformers import ElectraEmbeddings
# elec_emb= ElectraEmbeddings()


In [99]:
display(model.embeddings)
model.embeddings.position_embeddings

# help(model.embeddings)#.position_embeddings

XLMRobertaEmbeddings(
  (word_embeddings): Embedding(250002, 1024, padding_idx=1)
  (position_embeddings): Embedding(514, 1024, padding_idx=1)
  (token_type_embeddings): Embedding(1, 1024)
  (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

Embedding(514, 1024, padding_idx=1)

In [100]:
text= data['sentence'][1]
label= data['labels'][1]
text= preprocessing(text)
inputs= tokenizer(text, padding=True, truncation=True, max_length= 128, is_split_into_words= True, return_tensors="pt")


embed= model.embeddings
x1= embed(input_ids= inputs['input_ids'])
x1.shape
# input_ids, token_type_ids=None, position_ids=None, mask = None

x2= model(input_ids= inputs['input_ids'])['last_hidden_state']
x2.shape

y1= torch.cat((x1, x2), dim=-1)#.shape


In [101]:
linear= nn.Linear(model_config.hidden_size*2, CONFIG.num_labels)
linear(y1).shape

torch.Size([1, 26, 8])

In [102]:
# torch.tensor(inputs.word_ids())

# Testing Misc

In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet("data.parquet")
# df = df[:5000]
df.head()

,sentence,pun,modified_sentence,punctuations,tokens,labels,t_len,l_len,sanity,O,COMMA,DARI,QUESTION,EXCLAMATION,HYPHEN,SEMICOLON,COLON
0,"সাংবাদিকরা যখন তাকে আবার জিজ্ঞেস করে , এর অর্থ...",1,"সাংবাদিকরা যখন তাকে আবার জিজ্ঞেস করে, এর অর্থ...","[,, ?]","[সাংবাদিকরা, যখন, তাকে, আবার, জিজ্ঞেস, করে, এর...","[O, O, O, O, O, COMMA, O, O, O, O, O, O, O, O,...",16,16,1,14,1,0,1,0,0,0,0
1,"প্রাথমিক উক্তি যা-ই হোক না কেন , নৌকা বাইচের আ...",1,"প্রাথমিক উক্তি যা- ই হোক না কেন, নৌকা বাইচের ...","[-, ,, ।]","[প্রাথমিক, উক্তি, যা, ই, হোক, না, কেন, নৌকা, ব...","[O, O, HYPHEN, O, O, O, COMMA, O, O, O, O, O, ...",16,16,1,13,1,1,0,0,1,0,0
2,"তিনি বর্ণনা করেন: ""দীর্ঘ সময় ধরে, মানুষের পূর্...",1,"তিনি বর্ণনা করেন: দীর্ঘ সময় ধরে, মানুষের পূর্...","[:, ,, ,, ।]","[তিনি, বর্ণনা, করেন, দীর্ঘ, সময়, ধরে, মানুষের,...","[O, O, COLON, O, O, COMMA, O, O, O, O, O, O, O...",22,22,1,18,2,1,0,0,0,0,1
3,পতৌদি যদি আমি সেটা না করি ?,1,পতৌদি যদি আমি সেটা না করি?,[?],"[পতৌদি, যদি, আমি, সেটা, না, করি]","[O, O, O, O, O, QUESTION]",6,6,1,5,0,0,1,0,0,0,0
4,ঊষর আন্দিজের খনি আর কৃষিজমি সম্বল করে গড়ে ওঠা ...,1,ঊষর আন্দিজের খনি আর কৃষিজমি সম্বল করে গড়ে ওঠা ...,"[,, ,, -, ।]","[ঊষর, আন্দিজের, খনি, আর, কৃষিজমি, সম্বল, করে, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",29,29,1,25,2,1,0,0,1,0,0


In [3]:
# puns = []
# labs = []
# for i, row in df.iterrows():
#     for l in row['labels']:
#         if l not in labs:
#             labs.append(l)
# print(labs)

In [4]:
from dataset import prepare_dataloader
train_df = df[:int(len(df)*0.8)]
valid_df = df[int(len(df)*0.8):].reset_index

train_loader, valid_loader = prepare_dataloader(train_df, valid_df)

/home/virus_proton/Projects/P_Projects/Punctuation/Bengali_Punctuation/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import configuration
CONFIG = configuration.CONFIG()
CONFIG.label2id

{'O': 0,
 'COMMA': 1,
 'DARI': 2,
 'QUESTION': 3,
 'EXCLAMATION': 4,
 'HYPHEN': 5,
 'SEMICOLON': 6,
 'COLON': 7}

In [6]:
for data in train_loader:
    print(data)
    break

# data['input_ids'][0]
from transformers import AutoTokenizer
CONFIG.tokenizer = AutoTokenizer.from_pretrained(CONFIG.model_name)


{'input_ids': tensor([[     0,  36827,  14403, 130327,   3495,  50007,  37353,  14510, 240173,
         207867,  32368,   8619,  26020,  52797,   4822, 135524, 113547,  11622,
              6,  41758,   2801,  47835,      2,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1],
        [     0, 176749, 203055,      2,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1],
        [     0,  62444,   4876,  16209,  19293,  70904, 107913,  45397,  37564,
          54276, 177050,  58851,  26061,  6

In [7]:
# from Model import TokenClassificationModel

# model = TokenClassificationModel()
# model

In [8]:
# model.eval()
# import torch

# with torch.no_grad():
#     out = model(**data)

# out[0].shape
# out= out.detach().cpu().numpy().argmax(axis= -1)
# out.shape

In [10]:
from train import main

main(train_df, valid_df)

TypeError: fetch_scheduler() got an unexpected keyword argument 'optmizer'